In [ ]:
cd ..

In [ ]:
import echofilter.raw

In [ ]:
import os

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
top_color = 'c'
bot_color = '#00ee00'

In [ ]:
ROOT_DATA_DIR = '/media/scott/scratch/Datasets/dsforce/'
sample = 'surveyExports/mobile/Survey17/Survey17_GR4_T1W_E'

fname_raw = os.path.join(ROOT_DATA_DIR, sample + '_Sv_raw.csv')
fname_masked = os.path.join(ROOT_DATA_DIR, sample + '_Sv.csv')

ts_raw, depths_raw, signals_raw = echofilter.raw.loader.transect_loader(fname_raw)
ts_mskd, depths_mskd, signals_mskd = echofilter.raw.loader.transect_loader(fname_masked)

t_bot, d_bot = echofilter.raw.loader.evl_loader(os.path.join(ROOT_DATA_DIR, sample + '_bottom.evl'))
t_top, d_top = echofilter.raw.loader.evl_loader(os.path.join(ROOT_DATA_DIR, sample + '_turbulence.evl'))

In [ ]:
ts_new, d_top_new, d_bot_new = echofilter.raw.manipulate.make_lines_from_masked_csv(fname_masked)

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_mskd, depths_mskd, signals_mskd.T)
plt.plot(ts_new, d_top_new, top_color)
plt.plot(ts_new, d_bot_new, bot_color)
plt.gca().invert_yaxis()
plt.show()